<b>Loading data</b>

In [1]:
import torch
from torch import nn 

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.FashionMNIST(root="data", train=True, transform=ToTensor(), download=True)
test_data = datasets.FashionMNIST(root="data", train=False, transform=ToTensor(), download=True)

from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)


<b>Making Model</b>

In [ ]:
class FashionMNISTV2(nn.module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self):
        pass